In [6]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# 15년도의 데이터를 기준으로 left join 하여 합친 데이터프레임
# 통합 기준 : CUSTNO
dff_1415 = pd.read_pickle('dataset/fall1415.pkl')
drop_column = ['PURCHDATE', 'PURCHTIME', 'AFFIL', 'CLASSCODEM', 'CLASSCODES',
              'GENDER', 'STORENO', 'RESIDENCE', 'AGEGROUP']

# 재구매율만 보기위해 불필요한 컬럼 제거 
dff_1415.drop(drop_column, axis=1, inplace=True)
dff_1415

,RECEIPTNO,CUSTNO,CLASSCODEL,PURCHAMOUNT,Year,AFFIL_CLASS
0,08068631,07122,8,14000,2014,B8
1,08942410,17967,15,1750,2014,B15
2,08942410,17967,47,7900,2014,B47
3,08942410,17967,77,16898,2014,B77
4,08942410,17967,82,7731,2014,B82
...,...,...,...,...,...,...
3449415,07143341,08054,45,15800,2015,B45
3449416,07143342,05446,18,2000,2015,B18
3449417,07598192,01584,90,11850,2015,B90
3449418,07598193,00033,5,3160,2015,B5


In [4]:
len(dff_1415.AFFIL_CLASS.unique())
dff_1415.AFFIL_CLASS.unique()

NameError: name 'dff_1415' is not defined

In [10]:
# 구매 빈도수 하위 20개 품목 우선 선정
dff_1415.groupby('AFFIL_CLASS').count().sort_values(by=['CUSTNO'],
                                                    ascending=False).head(20)

,RECEIPTNO,CUSTNO,CLASSCODEL,PURCHAMOUNT,Year
AFFIL_CLASS,,,,,
A1,845966,845966,845966,845966,845966
C3,421373,421373,421373,421373,421373
C15,307543,307543,307543,307543,307543
B10,272409,272409,272409,272409,272409
B14,251929,251929,251929,251929,251929
C7,213457,213457,213457,213457,213457
B5,176732,176732,176732,176732,176732
A4,176154,176154,176154,176154,176154
C11,167189,167189,167189,167189,167189


In [4]:
dff_1415.groupby('AFFIL_CLASS').count().sort_values(by=['CUSTNO'],
                                                    ascending=False)

,RECEIPTNO,CUSTNO,CLASSCODEL,PURCHAMOUNT,Year
AFFIL_CLASS,,,,,
A1,845966,845966,845966,845966,845966
C3,421373,421373,421373,421373,421373
C15,307543,307543,307543,307543,307543
B10,272409,272409,272409,272409,272409
B14,251929,251929,251929,251929,251929
...,...,...,...,...,...
B85,246,246,246,246,246
D7,166,166,166,166,166
B42,26,26,26,26,26


### 재구매율을 구하기위해 사용한 코드

 - 돌리면 엄청 오래걸림

In [5]:
# # 기존 데이터프레임을 제품군별로 groupby 한 후 구매 횟수대로 내림차순 정렬. 하위 60개의 제품 추출
 tail60 = dff_1415.groupby('AFFIL_CLASS').count().sort_values(by=['CUSTNO'],
                                                     ascending=False).tail(60).index

# # 하위 제품군 60개에 대해 반복문을 실행하여 14년 15년 모두 특정 제품을 재구매한 고객 count
 for prod in tail60:
    
    pur_cnt = 0  # 14년, 15년 모두 제품을 구매한 고객을 count 하기 위한 변수
    tail_df = dff_1415.loc[dff_1415.AFFIL_CLASS==prod] # 하위 60개 중 하나에 해당하는 제품
    total_pur_cnt = len(tail_df.CUSTNO.unique()) # 해당 제품을 구매한 전체 고객수
    cust_index = tail_df.CUSTNO.unique() # 해당 제품을 구매한 고객의 index
    
    # 반복문 변수 prod에 해당하는 제품을 구매한 고객 중 재구매가 이루어진 고객을 찾기위한 반복문
    for cust in cust_index:        
        _case1 = (tail_df.CUSTNO==cust) & (tail_df.AFFIL_CLASS==prod) & \
        (tail_df.Year==2014) # 해당 재품을 구매한 고객 중 2014년에 구매한 고객 조걱
        _case2 = (tail_df.CUSTNO==cust) & (tail_df.AFFIL_CLASS==prod) & \
        (tail_df.Year==2015) # 해당 재품을 구매한 고객 중 2015년에 구매한 고객 조걱
        
        #만약 재구매가 이루어졌다면 pur_cnt변수에 1을 더해줌
        if (len(tail_df.loc[_case1])>0) & (len(tail_df.loc[_case2])>0):
            pur_cnt += 1
        
    # 해당 제품의 제품명, 재구매 고객 수, 전체 구매 고객 수 대비 재구매한 고객 비율
    print('제품명 : {0}, 재구매고객수 : {1} 1년간 재구매율 : {2}%'.\
          format(prod, pur_cnt, round((pur_cnt/total_pur_cnt)*100, 2)))
    

### 결과값 

제품명 : B41, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B40, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B42, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : D7, 재구매고객수 : 7 1년간 재구매율 : 5.93%
제품명 : B85, 재구매고객수 : 4 1년간 재구매율 : 1.83%
제품명 : B60, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : D3, 재구매고객수 : 81 1년간 재구매율 : 20.93%
제품명 : B58, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B35, 재구매고객수 : 21 1년간 재구매율 : 2.32%
제품명 : D5, 재구매고객수 : 75 1년간 재구매율 : 18.56%
제품명 : B44, 재구매고객수 : 140 1년간 재구매율 : 10.3%
제품명 : D6, 재구매고객수 : 141 1년간 재구매율 : 19.18%
제품명 : D8, 재구매고객수 : 119 1년간 재구매율 : 18.36%
제품명 : B69, 재구매고객수 : 164 1년간 재구매율 : 9.82%
제품명 : B32, 재구매고객수 : 182 1년간 재구매율 : 15.27%
제품명 : B50, 재구매고객수 : 204 1년간 재구매율 : 15.36%
제품명 : B70, 재구매고객수 : 197 1년간 재구매율 : 10.87%
제품명 : B88, 재구매고객수 : 211 1년간 재구매율 : 11.78%
제품명 : B49, 재구매고객수 : 207 1년간 재구매율 : 15.52%
제품명 : B36, 재구매고객수 : 226 1년간 재구매율 : 10.93%
제품명 : B61, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B59, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B23, 재구매고객수 : 306 1년간 재구매율 : 12.24%
제품명 : B71, 재구매고객수 : 404 1년간 재구매율 : 18.45%
제품명 : D1, 재구매고객수 : 215 1년간 재구매율 : 23.84%
제품명 : B25, 재구매고객수 : 449 1년간 재구매율 : 21.29%
제품명 : B22, 재구매고객수 : 345 1년간 재구매율 : 10.25%
제품명 : B51, 재구매고객수 : 532 1년간 재구매율 : 19.06%
제품명 : B78, 재구매고객수 : 620 1년간 재구매율 : 20.95%
제품명 : D2, 재구매고객수 : 277 1년간 재구매율 : 23.88%
제품명 : B30, 재구매고객수 : 595 1년간 재구매율 : 23.94%
제품명 : B62, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B75, 재구매고객수 : 544 1년간 재구매율 : 20.46%
제품명 : B86, 재구매고객수 : 609 1년간 재구매율 : 15.43%
제품명 : B77, 재구매고객수 : 768 1년간 재구매율 : 22.28%
제품명 : B28, 재구매고객수 : 697 1년간 재구매율 : 19.53%
제품명 : B76, 재구매고객수 : 874 1년간 재구매율 : 24.77%
제품명 : B2, 재구매고객수 : 772 1년간 재구매율 : 26.03%
제품명 : B45, 재구매고객수 : 920 1년간 재구매율 : 22.91%
제품명 : B64, 재구매고객수 : 800 1년간 재구매율 : 20.3%
제품명 : B3, 재구매고객수 : 1144 1년간 재구매율 : 27.17%
제품명 : B34, 재구매고객수 : 0 1년간 재구매율 : 0.0%
제품명 : B68, 재구매고객수 : 1334 1년간 재구매율 : 26.1%
제품명 : B84, 재구매고객수 : 1392 1년간 재구매율 : 24.78%
제품명 : B63, 재구매고객수 : 1487 1년간 재구매율 : 28.07%
제품명 : D4, 재구매고객수 : 251 1년간 재구매율 : 24.98%
제품명 : B31, 재구매고객수 : 1349 1년간 재구매율 : 37.05%
제품명 : B83, 재구매고객수 : 1554 1년간 재구매율 : 32.52%
제품명 : B54, 재구매고객수 : 1323 1년간 재구매율 : 26.41%
제품명 : A7, 재구매고객수 : 1541 1년간 재구매율 : 31.01%
제품명 : B87, 재구매고객수 : 1958 1년간 재구매율 : 29.91%
제품명 : B27, 재구매고객수 : 1037 1년간 재구매율 : 33.27%
제품명 : B4, 재구매고객수 : 2262 1년간 재구매율 : 38.65%
제품명 : B80, 재구매고객수 : 2186 1년간 재구매율 : 36.15%
제품명 : B72, 재구매고객수 : 2025 1년간 재구매율 : 34.76%
제품명 : A8, 재구매고객수 : 1696 1년간 재구매율 : 35.46%
제품명 : B53, 재구매고객수 : 1761 1년간 재구매율 : 26.9%
제품명 : B8, 재구매고객수 : 1606 1년간 재구매율 : 34.58%
제품명 : B57, 재구매고객수 : 2222 1년간 재구매율 : 37.42%
제품명 : B82, 재구매고객수 : 2134 1년간 재구매율 : 38.72%
제품명 : C2, 재구매고객수 : 2587 1년간 재구매율 : 54.75%
제품명 : B9, 재구매고객수 : 2226 1년간 재구매율 : 39.01%
제품명 : B29, 재구매고객수 : 2439 1년간 재구매율 : 42.22%
제품명 : B7, 재구매고객수 : 2685 1년간 재구매율 : 35.87%
제품명 : B81, 재구매고객수 : 3131 1년간 재구매율 : 43.05%
제품명 : B37, 재구매고객수 : 1607 1년간 재구매율 : 35.27%
제품명 : B56, 재구매고객수 : 2222 1년간 재구매율 : 39.47%
제품명 : B66, 재구매고객수 : 3461 1년간 재구매율 : 43.73%
제품명 : B33, 재구매고객수 : 2953 1년간 재구매율 : 40.57%
제품명 : B55, 재구매고객수 : 2832 1년간 재구매율 : 39.08%
제품명 : B26, 재구매고객수 : 2252 1년간 재구매율 : 44.37%
제품명 : B73, 재구매고객수 : 3580 1년간 재구매율 : 46.86%
제품명 : B46, 재구매고객수 : 3628 1년간 재구매율 : 48.92%
제품명 : A5, 재구매고객수 : 3681 1년간 재구매율 : 45.61%
제품명 : B79, 재구매고객수 : 3641 1년간 재구매율 : 48.68%
제품명 : B24, 재구매고객수 : 3439 1년간 재구매율 : 45.36%
제품명 : B47, 재구매고객수 : 4120 1년간 재구매율 : 51.2%
제품명 : B13, 재구매고객수 : 4703 1년간 재구매율 : 52.49%
제품명 : B92, 재구매고객수 : 4129 1년간 재구매율 : 50.34%
제품명 : B19, 재구매고객수 : 5210 1년간 재구매율 : 55.03%
제품명 : C10, 재구매고객수 : 3398 1년간 재구매율 : 61.31%
제품명 : B6, 재구매고객수 : 4354 1년간 재구매율 : 55.66%
제품명 : B65, 재구매고객수 : 4429 1년간 재구매율 : 50.86%
제품명 : B90, 재구매고객수 : 4169 1년간 재구매율 : 53.46%
제품명 : A3, 재구매고객수 : 3108 1년간 재구매율 : 51.88%
제품명 : B89, 재구매고객수 : 5507 1년간 재구매율 : 61.89%
제품명 : B17, 재구매고객수 : 5555 1년간 재구매율 : 57.82%
제품명 : B67, 재구매고객수 : 4788 1년간 재구매율 : 52.09%
제품명 : B48, 재구매고객수 : 5184 1년간 재구매율 : 60.67%
제품명 : B12, 재구매고객수 : 5402 1년간 재구매율 : 54.66%
제품명 : B18, 재구매고객수 : 6302 1년간 재구매율 : 62.4%
제품명 : C9, 재구매고객수 : 3624 1년간 재구매율 : 60.17%
제품명 : A9, 재구매고객수 : 5507 1년간 재구매율 : 60.19%
제품명 : C6, 재구매고객수 : 3660 1년간 재구매율 : 61.61%
제품명 : A6, 재구매고객수 : 5763 1년간 재구매율 : 53.71%
제품명 : B91, 재구매고객수 : 5527 1년간 재구매율 : 58.79%
제품명 : C14, 재구매고객수 : 4422 1년간 재구매율 : 65.88%
제품명 : B16, 재구매고객수 : 5015 1년간 재구매율 : 56.39%
제품명 : B52, 재구매고객수 : 6664 1년간 재구매율 : 64.6%
제품명 : B21, 재구매고객수 : 6202 1년간 재구매율 : 60.06%
제품명 : C4, 재구매고객수 : 4333 1년간 재구매율 : 65.73%
제품명 : C8, 재구매고객수 : 4307 1년간 재구매율 : 65.24%
제품명 : B1, 재구매고객수 : 6317 1년간 재구매율 : 63.36%
제품명 : C13, 재구매고객수 : 4662 1년간 재구매율 : 65.97%
제품명 : C5, 재구매고객수 : 4185 1년간 재구매율 : 65.97%
제품명 : B20, 재구매고객수 : 8162 1년간 재구매율 : 69.55%
제품명 : B11, 재구매고객수 : 7637 1년간 재구매율 : 69.28%
제품명 : C16, 재구매고객수 : 4892 1년간 재구매율 : 65.66%
제품명 : B38, 재구매고객수 : 7198 1년간 재구매율 : 68.78%
제품명 : B74, 재구매고객수 : 8129 1년간 재구매율 : 71.59%
제품명 : B15, 재구매고객수 : 7158 1년간 재구매율 : 64.31%
제품명 : C1, 재구매고객수 : 5062 1년간 재구매율 : 63.28%
제품명 : A2, 재구매고객수 : 8830 1년간 재구매율 : 70.54%
제품명 : C17, 재구매고객수 : 5563 1년간 재구매율 : 63.07%
제품명 : B43, 재구매고객수 : 8012 1년간 재구매율 : 71.07%
제품명 : C12, 재구매고객수 : 3556 1년간 재구매율 : 57.86%
제품명 : C11, 재구매고객수 : 5032 1년간 재구매율 : 61.07%
제품명 : A4, 재구매고객수 : 9157 1년간 재구매율 : 69.98%
제품명 : B5, 재구매고객수 : 7789 1년간 재구매율 : 69.61%
제품명 : C7, 재구매고객수 : 5518 1년간 재구매율 : 64.95%
제품명 : B14, 재구매고객수 : 9015 1년간 재구매율 : 72.25%
제품명 : B10, 재구매고객수 : 9055 1년간 재구매율 : 73.13%
제품명 : C15, 재구매고객수 : 5838 1년간 재구매율 : 63.24%
제품명 : C3, 재구매고객수 : 5880 1년간 재구매율 : 66.28%
제품명 : A1, 재구매고객수 : 10521 1년간 재구매율 : 75.96%

In [ ]:
# 재구매율 0%인 제품
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B34'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B62'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B59'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B61'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B58'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B60'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B42'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B40'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B41'].index, axis=0)
ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B85'].index, axis=0) 
# 위에 B85는 재구매고객수4, 재구매율이 1.8%의 가전이 품목 - 제외 하는게 좋을듯

# # 분류한 제품 중 left join으로 인해 재구매에서 제외된 제품들

# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B73'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B74'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B75'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B76'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B77'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B78'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B79'].index, axis=0)
# ndf = ndf.drop(ndf.loc[ndf.AFFIL_CLASS=='B80'].index, axis=0)

ndf

In [3]:
#  6가지의 제품군으로 재분류한 결과
# 각각 food(음식), nor(생활용품), cloth(의류), hobby(취미), rich(사치), etc(기타)
# 에 해당하는 코드 (제휴사 + 대분류코드)

total_food = ['A1','B1', 'B2', 'B3', 'B4', 'B5', 'B6',
          'B7', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16','B37', 'B38',
          'B43', 'B46', 'B47', 'B48','B52', 'B53', 'B54', 'B55',
          'B56', 'B57', 'B58', 'B59', 'B60', 'B61', 'B62', 'B63',
          'B64', 'B65', 'B67', 'B72','B73','B74','B75','B76','B77','B78','B79','B80','B81', 'B82', 'B83', 'B89', 'B91', 'B92',
         'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
         'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'D4']


total_nor = ['A2', 'A3', 'A7', 'A9', 'B17',
        'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B31',
         'B34', 'B66', 'B77', 'B84', 'B86', 'B87', 'B88',
             'C16', 'C17', 'D1', 'D2', 'D3', 'D5', 'D6']

total_cloth = ['A4', 'A5', 'B29','B30', 'B32', 'B33', 'B44',
               'B49', 'B50', 'B51', 'B68', 'B69']

total_hobby = ['A6', 'B25', 'B26', 'B27', 'B35', 'B36', 'B71']

total_rich = ['A8', 'B28', 'B70']
total_etc = ['B8', 'B45', 'B90', 'D7', 'D8']


# 위에서 분류한 것을 기준으로 mapping 진행
def prod_cat(data):
    if data in total_food:
        return 'food'
    elif data in total_nor:
        return 'nor'
    elif data in total_cloth:
        return 'cloth'
    elif data in total_hobby:
        return 'hobby'
    elif data in total_rich:
        return 'luxury'
    elif data in total_etc:
        return 'etc'
    
# 원본 DataFrame을 복사한 ndf 생성
ndf = dff_1415.copy()

# 제휴사(AFFIL) 와 대분류코드(CLASS)를 합친 column인 AFFIL_CLASS 를 바탕으로
# 새로운 제품분류 column인 prod_cat column 생성
ndf['prod_cat'] = ndf.AFFIL_CLASS.map(prod_cat)
ndf

NameError: name 'dff_1415' is not defined

<ndf DataFrame column 설명>

column 설명

- RECEIPTNO : 영수증 번호
- CUSTNO : 고객의 고유 번호
- PURCHDATE : 구매 날짜
- PURCHTIME : 구매 시간
- AFFIL : 제휴사
- CLASSCODEL : 대분류코드
- CLASSCODEM : 중분류코드
- CLASSCODES : 소분류코드
- STORENO : 점포코드
- GENDER : 성별
- AGEGROUP : 나이대
- RESIDENCE : 거주지역 코드
- PURCHAMOUNT : 해당 상품의 가격
- Year : 구매 년도
- AFFIL_CLASS : 제조사와 대분류코드를 합침
- prod_cat : AFFIL_CLASS를 바탕으로 food(식품), nor(생활용품), cloth(의류), hobby(취미), rich(사치), etc(기타)로 분류

In [8]:
# 제품군을 groupby 하여 묶은 후 2014년의 제품군 별 총 매출액 계산
_14 = ndf[ndf['Year']==2014].groupby('prod_cat').sum()[['PURCHAMOUNT']]

In [9]:
# 제품군을 groupby 하여 묶은 후 2015년의 제품군 별 총 매출액 계산
_15 = ndf[ndf['Year']==2015].groupby('prod_cat').sum()[['PURCHAMOUNT']]

In [10]:
# 15년의 제품군 별 총액을 _14에 15Purch라는 새로운 column을 생성하여 저장
_14['15Purch'] = _15.PURCHAMOUNT

In [11]:
_14

,PURCHAMOUNT,15Purch
prod_cat,,
cloth,27197183180,26989361870
etc,942513382,1061529189
food,19605646755,21971259865
hobby,7205198730,7006802540
luxury,7672905780,9218177080
nor,23053664374,23864326691


In [12]:
# 14년의 제품군 별 총 금액과 15년의 제품군 별 총 금액의 차이를 계산하여
# diff 라는 새로운 column에 저장
_14['diff'] = _14.apply(lambda x: x['15Purch'] - x['PURCHAMOUNT'], axis=1)

In [13]:
_14

,PURCHAMOUNT,15Purch,diff
prod_cat,,,
cloth,27197183180,26989361870,-207821310
etc,942513382,1061529189,119015807
food,19605646755,21971259865,2365613110
hobby,7205198730,7006802540,-198396190
luxury,7672905780,9218177080,1545271300
nor,23053664374,23864326691,810662317


In [14]:
# 제품군 중 의류와 취미 항목만 분류하여 
ndf.loc[(ndf.prod_cat=='cloth') | (ndf.prod_cat=='hobby')]

,RECEIPTNO,CUSTNO,CLASSCODEL,PURCHAMOUNT,Year,AFFIL_CLASS,prod_cat
383,02656975,16039,4,25200,2014,A4,cloth
390,10091726,11774,26,39000,2014,B26,hobby
493,02758141,19326,4,1570000,2014,A4,cloth
498,02758146,01088,4,300000,2014,A4,cloth
499,02758147,00775,4,39200,2014,A4,cloth
...,...,...,...,...,...,...,...
3449346,10721369,14793,25,8550,2015,B25,hobby
3449349,10721372,14793,26,10900,2015,B26,hobby
3449350,10721373,14492,68,5000,2015,B68,cloth
3449381,10721394,12054,26,17500,2015,B26,hobby
